In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

## Raw to subject-activity based acc/gyro files

In [ ]:
# some subjects don't have all the activities and some and data from different runs
activities = ['climbingdown', 'climbingup', 'jumping', 'lying', 'running', 'sitting', 'standing', 'walking',]
subject_activities = {
    'proband1':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband2':['climbingdown', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband3':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband4':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband5':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband6':['climbingdown', 'climbingup', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband7':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband8':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband9':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband10':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband11':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband12':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband13':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband14':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband15':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
}
sensor_locations = ['chest',
                    'forearm',
                    'head',
                    'shin',
                    'thigh',
                    'upperarm',
                    'waist',]
sensors = ['acc', 'gyr']
sensor_names = {'acc':'acc', 'gyr':'Gyroscope'}
subjects = [sub for sub in os.listdir('./data/realworld2016_dataset/') if 'proband' in sub]

In [ ]:
def find_reference_times(subject, activity):
    global sensors, sensor_locations, sensor_names
    times = []
    reference_sensors = []
    reference_sensor_locations = []
    root = './data/realworld2016_dataset/'+subject+'/data/'
    for sensor in sensors:
        for sensor_location in sensor_locations:
            path = root+sensor+'_'+activity+'_csv/'+sensor_names[sensor]+'_'+activity+'_'+sensor_location+'.csv'
            df = pd.read_csv(path)
            times.append(df['attr_time'].iloc[0])
            reference_sensors.append(sensor)
            reference_sensor_locations.append(sensor_location)
    idx = np.where(np.max(times) == times)[0][0]
    path = root+reference_sensors[idx]+'_'+activity+'_csv/'+sensor_names[reference_sensors[idx]]+'_'+activity+'_'+reference_sensor_locations[idx]+'.csv'
    df = pd.read_csv(path)
    reference_time = df['attr_time'].values[0]
    for sensor in sensors:
        for sensor_location in sensor_locations:
            path = root+sensor+'_'+activity+'_csv/'+sensor_names[sensor]+'_'+activity+'_'+sensor_location+'.csv'
            df = pd.read_csv(path)
            times.append(df['attr_time'].iloc[-1])
            reference_sensors.append(sensor)
            reference_sensor_locations.append(sensor_location)
    idx = np.where(np.min(times) == times)[0][0]
    path = root+reference_sensors[idx]+'_'+activity+'_csv/'+sensor_names[reference_sensors[idx]]+'_'+activity+'_'+reference_sensor_locations[idx]+'.csv'
    df = pd.read_csv(path)
    apocalypse = df['attr_time'].values[-1]
    return reference_time, apocalypse
    
class syncer:
    def __init__(self, time_ref):
        self.ts = time_ref
        self.T = 20
        
    def find_sync_point(self, times):
        diff = np.abs(times-self.ts)
        if np.min(diff)>2560:
            self.ts = self.ts+self.T
            return np.nan
        else:
            index = np.where(diff==np.min(diff))[0][0]
            self.ts = self.ts+self.T
            return index
    
def generate_columns(sensors, sensor_locations):
    columns = []
    axes = ['TS', 'x', 'y', 'z']
    for sensor_location in sensor_locations:
        for sensor in sensors:
            for axis in axes:
                columns.append(sensor+'_'+sensor_location+'_'+axis)
    return columns

In [ ]:
columns = generate_columns(sensors, sensor_locations)
for subject in subjects:
    for activity in subject_activities[subject]:
        time_reference, apocalypse = find_reference_times(subject, activity)
        length = int((apocalypse-time_reference)//20)
        root = './data/realworld2016_dataset/'+subject+'/data/'
        sensor_location_temp = []
        for sensor_location in sensor_locations:
            sensors_temp = []
            for sensor in sensors:
                sync = syncer(time_reference)
                path = root+sensor+'_'+activity+'_csv/'+sensor_names[sensor]+'_'+activity+'_'+sensor_location+'.csv'
                df = pd.read_csv(path)
                sensor_temp = []
                for idx in range(length):
                    sync_point = sync.find_sync_point(df['attr_time'].values)
                    if sync_point is np.nan:
                        nan_array = np.empty(df.iloc[0, 1:].values.shape)
                        nan_array[:] = np.nan
                        sensor_temp.append(nan_array)
                    else:
                        sensor_temp.append(df.iloc[sync_point, 1:].values)
                sensor_temp = np.stack(sensor_temp)
                sensors_temp.append(sensor_temp)
            sensors_temp = np.concatenate(sensors_temp, axis=1)
            sensor_location_temp.append(sensors_temp)
        sensor_location_temp = np.concatenate(sensor_location_temp, axis=1)
        save_path = './data/realworld2016_dataset/'+subject+'/processed/'+subject+'_'+activity+'.csv'
        storedf = pd.DataFrame(sensor_location_temp, columns = columns)
        storedf.to_csv(save_path)
        print('Subject: {}, Activity: {}, Saved to: {}'.format(subject, activity, save_path))

## Stacking splitted data

In [ ]:
# paths = ['G:/Projects/HAR/data/realworld2016_dataset/proband14/data/acc_climbingup_csv/acc_climbingup_waist.csv',
#          'G:/Projects/HAR/data/realworld2016_dataset/proband14/data/acc_climbingup_csv/acc_climbingup_2_waist.csv',
#          'G:/Projects/HAR/data/realworld2016_dataset/proband14/data/acc_climbingup_csv/acc_climbingup_3_waist.csv']
# columns = pd.read_csv(paths[0]).columns[1:].tolist()

In [ ]:
paths = ['./data/realworld2016_dataset/proband7/data/gyr_climbingup_csv/Gyroscope_climbingup_waist.csv',
         './data/realworld2016_dataset/proband7/data/gyr_climbingup_csv/Gyroscope_climbingup_2_waist.csv',
         './data/realworld2016_dataset/proband7/data/gyr_climbingup_csv/Gyroscope_climbingup_3_waist.csv']
columns = pd.read_csv(paths[0]).columns[1:].tolist()

In [ ]:
arr = np.concatenate((pd.read_csv(paths[0]).values[:, 1:], pd.read_csv(paths[1]).values[:, 1:], pd.read_csv(paths[2]).values[:, 1:]), axis=0)

In [ ]:
df = pd.DataFrame(arr, columns=columns)
df.to_csv(paths[0].split('/')[-1])

## NaN value check

In [ ]:
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py

In [ ]:
allfiles = [os.path.join(path, name) for path, subdirs, files in os.walk('./data/realworld2016_dataset/') for name in files if (('processed' in path) and ('desktop.ini' not in name))]

In [ ]:
# True ../data/realworld2016_dataset/proband1\processed\proband1_climbingup.csv
# True ../data/realworld2016_dataset/proband1\processed\proband1_standing.csv
# True ../data/realworld2016_dataset/proband14\processed\proband14_climbingdown.csv
# True ../data/realworld2016_dataset/proband14\processed\proband14_climbingup.csv
# True ../data/realworld2016_dataset/proband15\processed\proband15_sitting.csv
# True ../data/realworld2016_dataset/proband2\processed\proband2_running.csv
# True ../data/realworld2016_dataset/proband2\processed\proband2_walking.csv
# True ../data/realworld2016_dataset/proband4\processed\proband4_climbingdown.csv
# True ../data/realworld2016_dataset/proband4\processed\proband4_climbingup.csv
# True ../data/realworld2016_dataset/proband5\processed\proband5_climbingup.csv
# True ../data/realworld2016_dataset/proband7\processed\proband7_climbingdown.csv
# True ../data/realworld2016_dataset/proband7\processed\proband7_climbingup.csv

In [ ]:
pd.read_csv('../data/realworld2016_dataset/proband5\processed\proband5_climbingup.csv').isnull().sum()

## Subject/activity to window

In [ ]:
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
import tables

In [ ]:
activities = ['climbingdown', 'climbingup', 'jumping', 'lying', 'running', 'sitting', 'standing', 'walking',]
subject_activities = {
    'proband1':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband2':['climbingdown', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband3':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband4':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband5':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband6':['climbingdown', 'climbingup', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband7':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband8':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband9':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband10':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband11':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband12':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband13':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband14':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband15':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
}
paths = ['./data/realworld2016_dataset/proband3/processed/',
         './data/realworld2016_dataset/proband5/processed/',
         './data/realworld2016_dataset/proband6/processed/',
         './data/realworld2016_dataset/proband8/processed/',
         './data/realworld2016_dataset/proband9/processed/',
         './data/realworld2016_dataset/proband10/processed/',
         './data/realworld2016_dataset/proband11/processed/',
         './data/realworld2016_dataset/proband12/processed/',
         './data/realworld2016_dataset/proband13/processed/',
         './data/realworld2016_dataset/proband15/processed/',]
labels = {'climbingdown':0, 'climbingup':1, 'jumping':2, 'lying':3, 'running':4, 'sitting':5, 'standing':6, 'walking':7}
sensor_locations = ['chest',
                    'forearm',
                    'head',
                    'shin',
                    'thigh',
                    'upperarm',
                    'waist',]
sensors = ['acc', 'gyr']
sensor_names = {'acc':'acc', 'gyr':'Gyroscope'}
subjects = ['proband10',
            'proband11',
            'proband12',
            'proband13',
            'proband15',
            'proband3',
            'proband5',
            'proband6',
            'proband8',
            'proband9']

In [ ]:
def generate_data_columns(sensors, sensor_locations):
    columns = []
    axes = ['x', 'y', 'z']
    for sensor_location in sensor_locations:
        for sensor in sensors:
            for axis in axes:
                columns.append(sensor+'_'+sensor_location+'_'+axis)
    return columns

In [ ]:
w = 250
sr = 0.8 # about half of the data will be cut later and that will results in the same overlapping we used in PAMAP2
data_columns = generate_data_columns(sensors, sensor_locations)
s = int((1-sr)*w)
for subject in subjects:
    X = []
    y = []
    for activity in subject_activities[subject]:
        path = './data/realworld2016_dataset/'+subject+'/processed/'+subject+'_'+activity+'.csv'
        label = labels[activity]
        dataArr = pd.read_csv(path)[data_columns].iloc[750:-250, :].reset_index() # Skipping data where the subject is waiting for a signal to start performing the activity
        dataArr.interpolate(inplace=True, limit=100)
        for idx in range(0, dataArr.shape[0]-w, s):
            y.append(label)
            X.append(dataArr.iloc[idx:idx + w, :].values[:, 1:])
    X = np.stack(X)
    y = np.array(y)
    hdf_handle = tables.open_file('./data/realworld2016_dataset/'+subject+'/processed/'+subject+'-1000.hdf', 'w')
    dX = hdf_handle.create_array(hdf_handle.root, 'X', X)
    dy = hdf_handle.create_array(hdf_handle.root, 'y', y)
    hdf_handle.close()

### Subjects with missing data

In [ ]:
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
import tables

In [ ]:
def generate_data_columns(sensors, sensor_locations):
    columns = []
    axes = ['x', 'y', 'z']
    for sensor_location in sensor_locations:
        for sensor in sensors:
            for axis in axes:
                columns.append(sensor+'_'+sensor_location+'_'+axis)
    return columns

In [ ]:
activities = ['climbingdown', 'climbingup', 'jumping', 'lying', 'running', 'sitting', 'standing', 'walking',]
subject_activities = {
    'proband1':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband2':['climbingdown', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband3':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband4':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband5':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband6':['climbingdown', 'climbingup', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband7':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband8':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband9':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband10':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband11':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband12':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband13':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband14':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
    'proband15':['climbingdown', 'climbingup', 'jumping', 'lying',
                'running', 'sitting', 'standing', 'walking',],
}
labels = {'climbingdown':0, 'climbingup':1, 'jumping':2, 'lying':3, 'running':4, 'sitting':5, 'standing':6, 'walking':7}
sensor_locations = ['chest',
                    'forearm',
                    'head',
                    'shin',
                    'thigh',
                    'upperarm',
                    'waist',]
sensors = ['acc', 'gyr']
sensor_names = {'acc':'acc', 'gyr':'Gyroscope'}
subjects = ['proband14',
            'proband1',
            'proband2',
            'proband4',
            'proband7']
# 14, 1, 2, 4, 7

In [ ]:
w = 250
sr = 0.8
data_columns = generate_data_columns(sensors, sensor_locations)
s = int((1-sr)*w)
for subject in subjects:
    X = []
    y = []
    for activity in subject_activities[subject]:
        path = './data/realworld2016_dataset/'+subject+'/processed/'+subject+'_'+activity+'.csv'
        label = labels[activity]
        dataArr = pd.read_csv(path)[data_columns].iloc[1000:, :].reset_index()
        for idx in range(0, dataArr.shape[0]-w, s):
            if np.isnan(dataArr.iloc[idx:idx + w, :].values[:, 1:]).any():
                continue
            else:
                y.append(label)
                X.append(dataArr.iloc[idx:idx + w, :].values[:, 1:])
    X = np.stack(X)
    y = np.array(y)
    hdf_handle = tables.open_file('./data/realworld2016_dataset/'+subject+'/processed/'+subject+'-1000.hdf', 'w')
    dX = hdf_handle.create_array(hdf_handle.root, 'X', X)
    dy = hdf_handle.create_array(hdf_handle.root, 'y', y)
    hdf_handle.close()